In [2]:
import asyncio
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import random
import datetime
import sys
import os
import time

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

sys.path.append(os.path.abspath('../src'))
import funciones_auxiliares as fa

#---------------------------------------------------------------------------------------------------


async def pag_mes(munici):
    driver = webdriver.Chrome()
    url_wunder = "https://www.wunderground.com/history"
    driver.get(url_wunder)
    driver.maximize_window()
    sleep(1)
    #Denegamos las cookies
    iframe_cookies = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="sp_message_iframe_1165301"]')))
    driver.switch_to.frame(iframe_cookies)

    sleep(3)
    try:
        driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column").click()
    except:
        print("No encuentro el boton de las cookies")

    driver.switch_to.default_content()
    # Introduce el municipio en la página principal
    sleep(3)
    driver.find_element("css selector", "#historySearch").send_keys(f"{munici}, Spain", Keys.ENTER)

    # Damos click en view
    sleep(3)
    driver.find_element("css selector", "#dateSubmit").click()
    driver.find_element("css selector", "#dateSubmit").click()

    # Pinchar en monthly de la página del municipio
    sleep(2)
    driver.find_element("css selector", "#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)").click()

    sleep(3)
    url_mes = driver.find_element("xpath", "/html/head/link[51]").get_attribute("href")
    url_mes_base= url_mes[:-2]
    print(url_mes_base)
   
    df_final=pd.DataFrame()

    for i in range(1,11):
        url_mes = url_mes_base + f"{i}"
        driver.get(url_mes)
        sleep(3)

        # Cogemos el html de la pagina actual
        html_table_page = driver.page_source

        sopa = BeautifulSoup(html_table_page, "html.parser")
        df=fa.df_datos_mes(sopa)
        df_final = pd.concat([df_final, df])

    return df_final


async def main():
    inicio = time.time()
    
    municipios = ["aranjuez", "arroyomolinos"]
    lista_tareas = []
    for munici in municipios:
        lista_tareas.append(pag_mes(munici))

    lista_df_muni = await asyncio.gather(*lista_tareas)
    df_final = pd.concat(lista_df_muni)
    print(df_final.shape)

    df_final.reset_index(inplace=True)
    df_final.to_csv("../datos/df_asinc.csv")

    fin = time.time()
    tiempo_total = np.round(((fin - inicio)/60), 2)
    print(f"El tiempo total es {tiempo_total/60} min")


await main()


https://www.wunderground.com/history/monthly/LEMD/date/2024-
https://www.wunderground.com/history/monthly/LEMD/date/2024-
(582, 18)
El tiempo total es 0.036166666666666666 min


In [3]:
df = pd.read_csv("../datos/df_asinc.csv")

In [4]:
df

,Unnamed: 0,index,mes,dia,max_temp,avg_temp,min_temp,max_rocio,avg_rocio,min_rocio,max_humedad,avg_humedad,min_humedad,max_viento,avg_viento,min_viento,max_presion,avg_presion,min_presion,lluvia
0,0,0,Jan,1,50,41.0,37,41,37.4,34,100,87.3,62,8,2.7,0,28.1,28.0,28.0,0.0
1,1,1,Jan,2,45,40.3,34,43,38.3,32,100,93.2,81,5,1.6,0,28.1,28.1,28.0,0.0
2,2,2,Jan,3,55,50.4,43,54,48.9,43,100,94.9,88,13,4.5,0,28.0,28.0,27.9,0.0
3,3,3,Jan,4,50,48.0,46,50,47.5,46,100,98.2,93,10,2.4,0,28.0,27.9,27.7,0.0
4,4,4,Jan,5,50,45.7,41,46,37.5,32,93,74.0,54,18,8.7,1,27.8,27.7,27.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,577,12,Oct,13,75,63.6,55,59,55.9,54,100,78.7,53,6,1.8,0,28.0,28.0,27.9,0.0
578,578,13,Oct,14,73,64.5,57,61,58.5,55,100,81.9,61,6,1.0,0,28.0,27.9,27.8,0.0
579,579,14,Oct,15,70,64.4,61,61,59.1,55,100,83.0,64,20,7.9,0,27.8,27.7,27.7,0.0
580,580,15,Oct,16,73,65.0,61,63,58.6,55,94,81.3,53,20,11.9,2,27.8,27.7,27.6,0.0
